In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics

import os, time, itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

#### 경로확인 

In [2]:
os.getcwd()

'/Users/sungjinkim/Documents/Study/Regression_Project'

#### 데이터 준비 

In [3]:
batter = pd.read_csv('./Data/Regular_Season_Batter.csv')

batter['Age'] = batter['year'] - batter['year_born'].apply(lambda x: int(x[:4]))

batter['height/weight'] = batter['height/weight'].fillna('0cm/0kg')

batter['height'] = batter['height/weight'].apply(lambda x: int(x.split('/')[0][:-2]))
batter['weight'] = batter['height/weight'].apply(lambda x: int(x.split('/')[1][:-2]))

mean_h = np.mean(batter['height'][batter['height'] != 0])
mean_w = np.mean(batter['weight'][batter['weight'] != 0])

batter['height'] = batter['height'].apply(lambda x: mean_h if x == 0 else x)
batter['weight'] = batter['weight'].apply(lambda x: mean_w if x == 0 else x)

batter = batter[batter['AB'] >= 396]

batter = batter.drop(1935, axis = 0)

batter_conti = batter.drop(['batter_id', 'year', 'year_born', 'batter_name',
                            'team', 'position', 'career', 'starting_salary', 'height/weight'], axis=1)

#### Target 분리 

In [4]:
target_OPS = batter_conti['OPS']

batter_conti_ = batter_conti.drop(['avg', 'OPS', 'SLG', 'OBP'], axis=1)
batter_conti_1 = sm.add_constant(batter_conti_, has_constant='add')

#### 학습데이터 / 평가데이터 분할

In [5]:
feature_columns = list(batter_conti_1.columns)

X = batter_conti_1[feature_columns]
y = target_OPS
train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((332, 20), (143, 20), (332,), (143,))

### Baseline (Full) 모델 적합 

In [6]:
full_model = sm.OLS(train_y, train_x)
fitted_full_model = full_model.fit()
print(fitted_full_model.summary())

                            OLS Regression Results                            
Dep. Variable:                    OPS   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     2394.
Date:                Sat, 13 Jun 2020   Prob (F-statistic):          6.42e-323
Time:                        23:23:50   Log-Likelihood:                 1086.9
No. Observations:                 332   AIC:                            -2136.
Df Residuals:                     313   BIC:                            -2064.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8364      0.028     29.855      0.0

### 단계적 선택법을 통한 모델 적합

In [7]:
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)])
    regr = model.fit()
    AIC = regr.aic
    return {'model': regr, 'AIC': AIC}

def getBest(X, y, k):
    tic = time.time() # 시작시간
    results = [] # 결과 저장공간
    for combo in itertools.combinations(X.columns.difference(['const']), k): # 각 변수조합을 고려한 경우의 수
        combo = (list(combo) + ['const'])
        
        results.append(processSubset(X, y, feature_set=combo)) # 모델링된 것들을 저장
    models = pd.DataFrame(results)
    # 가장 낮은 AIC를 가지는 모델 선택 및 저장
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print('Processed', models.shape[0], 'models on', k, 'predictors in', (toc - tic), 'seconds')
    return best_model

def forward(X, y, predictors):
    # 데이터 변수들이 미리정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y= y, feature_set=predictors+[p]+['const']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)

    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[0] )
    return best_model

def backward(X,y,predictors):
    tic = time.time()
    results = []
    # 데이터 변수들이 미리정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X=X, y= y,feature_set=list(combo)+['const']))
    models = pd.DataFrame(results)
    # 가장 낮은 AIC를 가진 모델을 선택
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors) - 1, "predictors in",
          (toc - tic))
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[0] )
    return best_model

def Stepwise_model(X, y):
    Stepmodels = pd.DataFrame(columns=['AIC', 'model'])
    tic = time.time()
    predictors = []
    Smodel_before = processSubset(X, y, predictors+['const'])['AIC']
    # 변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(X.columns.difference(['const']))+1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        print('forward')
        Stepmodels.loc[i] = Forward_result
        predictors = Stepmodels.loc[i]['model'].model.exog_names
        predictors = [k for k in predictors if k != 'const']
        Backward_result = backward(X=X, y=y, predictors=predictors)
        if Backward_result['AIC'] < Forward_result['AIC']:
            Stepmodels.loc[i] = Backward_result
            predictors = Stepmodels.loc[i]['model'].model.exog_names
            Smodel_before = Stepmodels.loc[i]['AIC']
            predictors = [k for k in predictors if k != 'const']
            print('backward')
        if Stepmodels.loc[i]['AIC'] > Smodel_before: break
        else: Smodel_before = Stepmodels.loc[i]['AIC']
        
    toc = time.time()
    print('Total elapsed time:', (toc - tic), 'seconds')
    
    return (Stepmodels['model'][len(Stepmodels['model'])])

In [8]:
Stepwise_best_model = Stepwise_model(X=X, y=y)

Processed  19 models on 1 predictors in 0.023016929626464844
Selected predictors: ['TB', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8472de240>
forward
Processed  1 models on 0 predictors in 0.0017590522766113281
Selected predictors: ['const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8472b37f0>
Processed  18 models on 2 predictors in 0.025189876556396484
Selected predictors: ['TB', 'AB', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8472b3400>
forward
Processed  2 models on 1 predictors in 0.0038230419158935547
Selected predictors: ['TB', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8665a7eb8>
Processed  17 models on 3 predictors in 0.02134990692138672
Selected predictors: ['TB', 'AB', 'H', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8472a9cc0>
forward
Proces

Processed  11 models on 10 predictors in 0.016048192977905273
Selected predictors: ['TB', 'AB', 'H', 'BB', 'HBP', 'G', 'RBI', 'GDP', 'height', 'weight', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8472a9630>
backward
Processed  9 models on 11 predictors in 0.015167951583862305
Selected predictors: ['TB', 'AB', 'H', 'BB', 'HBP', 'G', 'RBI', 'GDP', 'height', 'weight', 'CS', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8665b2400>
forward
Processed  11 models on 10 predictors in 0.015181779861450195
Selected predictors: ['TB', 'AB', 'H', 'BB', 'HBP', 'G', 'RBI', 'GDP', 'height', 'weight', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fe8665b26a0>
backward
Total elapsed time: 0.6473391056060791 seconds


In [9]:
print(Stepwise_best_model.summary())

                            OLS Regression Results                            
Dep. Variable:                    OPS   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     5893.
Date:                Sat, 13 Jun 2020   Prob (F-statistic):               0.00
Time:                        23:23:50   Log-Likelihood:                 1545.2
No. Observations:                 475   AIC:                            -3068.
Df Residuals:                     464   BIC:                            -3023.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TB             0.0022   2.83e-05     77.551      0.0

In [10]:
mse_full = metrics.mean_squared_error(test_y, fitted_full_model.predict(test_x))
mse_step = metrics.mean_squared_error(test_y, Stepwise_best_model.predict(test_x[Stepwise_best_model.model.exog_names]))
print(f'mse_full: {mse_full}')
print(f'mse_Stepwise: {mse_step}')

mse_full: 0.00010025794675555962
mse_Stepwise: 9.116068912965468e-05


# MLP 모델 적합

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import *

## Hyperparameter 

In [12]:
num_epochs = 100
batch_size = 2

## Model 

In [13]:
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.dense1 = Dense(1024, activation='relu')
        self.bn1 = BatchNormalization()
        self.dense2 = Dense(1024, activation='relu')
        self.bn2 = BatchNormalization()
        self.dense3 = Dense(10, activation='relu')
        self.dense4 = Dense(1)
        
    def call(self, x, training=False):
        x = self.dense1(x)
        x = self.bn1(x, training=training)
        x = tf.keras.layers.Dropout(.5)(x, training=training)
        x = self.dense2(x)
        x = self.bn2(x, training=training)
        x = tf.keras.layers.Dropout(.5)(x, training=training)
        x = self.dense3(x)
        x = self.dense4(x)
        return x

In [14]:
train_x.shape, train_y.shape

((332, 20), (332,))

In [15]:
train_ds = tf.data.Dataset.from_tensor_slices((np.array(train_x), np.array(train_y))).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((np.array(test_x), np.array(test_y))).batch(batch_size)

In [16]:
@tf.function
def train_step(model, datas, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(datas, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, datas, labels, loss_object, test_loss, test_accuracy):
    predictions = model(datas, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## Define model, loss ftn, optimizer, metric 

In [17]:
model = MLP()

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanSquaredError(name='test_accuracy')

## Training 

In [18]:
for epoch in range(num_epochs):
    for datas, labels in train_ds:
        train_step(model, datas, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for datas, labels in test_ds:
        test_step(model, datas, labels, loss_object, test_loss, test_accuracy)
    
    template = 'Epoch {}, Loss: {}, Accuracy {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result(),
          test_loss.result(),
          test_accuracy.result()
    ))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 2.2134523391723633, Accuracy 2.2134523391723633, Test Loss: 9.269461631774902, Test Accuracy: 9.269461631774902
Epoch 2, Loss: 1.5531426668167114, Accuracy 1.5531426668167114, Test Loss: 6.401004314422607, Test Accuracy: 6.401004314422607
Epoch 3, Loss: 1.2762360572814941, Accuracy 1.2762360572814941, Test Loss: 5.746273994445801, Test Accuracy: 5.746273994445801
Epoch 4, Loss: 1.0688445568084717, Accuracy 1.0688445568084717, Test Loss: 5.377772331237793, Test Accuracy: 5.377772331237793


Epoch 59, Loss: 0.0999542772769928, Accuracy 0.0999542772769928, Test Loss: 0.7105911374092102, Test Accuracy: 0.7105911374092102
Epoch 60, Loss: 0.09838372468948364, Accuracy 0.09838372468948364, Test Loss: 0.6997751593589783, Test Accuracy: 0.6997751593589783
Epoch 61, Loss: 0.09686314314603806, Accuracy 0.09686314314603806, Test Loss: 0.6888818144798279, Test Accuracy: 0.6888818144798279
Epoch 62, Loss: 0.0953831672668457, Accuracy 0.0953831672668457, Test Loss: 0.6784348487854004, Test Accuracy: 0.6784348487854004
Epoch 63, Loss: 0.09396767616271973, Accuracy 0.09396767616271973, Test Loss: 0.6680673360824585, Test Accuracy: 0.6680673360824585
Epoch 64, Loss: 0.0925804153084755, Accuracy 0.0925804153084755, Test Loss: 0.6579773426055908, Test Accuracy: 0.6579773426055908
Epoch 65, Loss: 0.0912475734949112, Accuracy 0.0912475734949112, Test Loss: 0.6482732892036438, Test Accuracy: 0.6482732892036438
Epoch 66, Loss: 0.0899553969502449, Accuracy 0.0899553969502449, Test Loss: 0.638922

In [19]:
np.mean(test_y)

0.8529148882223112

In [20]:
np.mean(model(np.array(test_x)[:]))

0.9682022

In [21]:
np.mean(fitted_full_model.predict(test_x))

0.8514077739500004

In [22]:
np.mean(Stepwise_best_model.predict(test_x[Stepwise_best_model.model.exog_names]))

0.8516730151989939

In [23]:
print(test_y[:10], model(np.array(test_x)[:10]),
     fitted_full_model.predict(test_x)[:10],
     Stepwise_best_model.predict(test_x[Stepwise_best_model.model.exog_names])[:10])

1644    0.812000
165     0.888977
303     0.831000
917     0.774000
706     0.891000
1420    0.645000
2403    0.974531
314     0.822000
48      0.900000
853     0.938000
Name: OPS, dtype: float64 tf.Tensor(
[[1.4454021 ]
 [0.81549793]
 [0.7762311 ]
 [0.77965975]
 [0.92071223]
 [0.7581382 ]
 [0.9476874 ]
 [0.7762311 ]
 [0.80711335]
 [1.0202705 ]], shape=(10, 1), dtype=float32) 1644    0.801860
165     0.886882
303     0.823222
917     0.774614
706     0.897627
1420    0.624548
2403    0.964871
314     0.822992
48      0.896128
853     0.929856
dtype: float64 1644    0.804033
165     0.887479
303     0.823681
917     0.775409
706     0.895690
1420    0.626991
2403    0.963630
314     0.823280
48      0.896554
853     0.929271
dtype: float64
